In [1]:
%load_ext watermark
%watermark -a 'NavinKumarMNK' -v -p torch

Author: NavinKumarMNK

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.8.0

torch: 1.13.1



In [1]:
# Import the required modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchvision import models
import wandb
import torchmetrics

In [5]:
model = models.efficientnet_b3(include_top=False, weights='EfficientNet_B3_Weights.DEFAULT')
# remove last layer
model.classifier = nn.Identity()
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [4]:
torch.save(model, '../weights/EfficientNetb3Encoder.pt')

In [6]:
model_1 = torch.load('../weights/EfficientNetb3Encoder.pt')

In [17]:
# Decoder to construct the image again
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(1536, 1536*4*4)
        self.model = nn.Sequential(
            nn.ConvTranspose2d(1536, 1024, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 1536, 4, 4)
        return self.model(x)

In [18]:
input = torch.randn(16, 3, 256, 256)
encoder = model_1
decoder = Decoder()

feature = encoder(input)
print(feature.shape)
output = decoder(feature)
print(output.shape)
# differnece between input and output

model = decoder
total_params = sum(p.numel() for p in model.parameters())
print(total_params)


torch.Size([16, 1536])
torch.Size([16, 3, 256, 256])
58203651


In [20]:
torch.save(model.state_dict(), '../weights/EfficientNetb3DecoderLarge.pt', )

In [22]:
# classifier
class EfficientNetb3Classifier(nn.Module):
    def __init__(self, no_of_classes):
        super(EfficientNetb3Classifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(1536, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, no_of_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.model(x)


In [23]:
import sys
import os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))
from utils import utils

In [24]:
classes = int(utils.config_parse('GENERAL')['no_of_classes'])
classes

14

In [25]:

model = EfficientNetb3Classifier(classes)
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

EfficientNetb3Classifier(
  (model): Sequential(
    (0): Linear(in_features=1536, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=14, bias=True)
    (5): Softmax(dim=1)
  )
)
854414


In [26]:
torch.save(model.state_dict(), '../weights/EfficientNetb3Classifier.pt')